In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})
import numpy as np
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from parse_density_files import CUBE
import alchemy_tools as at

In [2]:
p38 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_38.cube'
p30 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_30.cube'
p23 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_23.cube'
p15 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_15.cube'
p8 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_8.cube'
p0 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/free_electron_gas.cube'
cubes = [(p0, 0), (p8, 8/38), (p15, 15/38), (p23, 23/38), (p30, 30/38), (p38, 1)]
# ae, alchpots = atomic_energy_decomposition(cubes)
trapz = at.atomic_energy_decomposition(cubes)
cpsline = at.atomic_energy_decomposition(cubes, intgr_method='cspline')

KeyboardInterrupt: 

In [ ]:
trapz[1]

In [ ]:
cpsline[1]

In [ ]:
trapz[1]-cpsline[1]

In [3]:
ve38 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_000227/cube-files/ve_38.cube')
ve30 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_000227/cube-files/ve_30.cube')
ve23 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_000227/cube-files/ve_23.cube')
ve15 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_000227/cube-files/ve_15.cube')
ve8 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_000227/cube-files/ve_8.cube')
ve0 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/ueg/ueg.cube')

In [6]:
lambda_densities = [ve0, ve8, ve15, ve23, ve30, ve38]
density_arrays = []
for dens in lambda_densities:
    dens.scale()
    density_arrays.append(dens.data_scaled)
lam_vals = [0, 8/38, 15/38, 23/38, 30/38, 1]

In [ ]:
density_arrays[1].shape
densities = np.array([ve0.data_scaled, ve8.data_scaled, ve15.data_scaled, ve23.data_scaled, ve30.data_scaled, ve38.data_scaled])

In [ ]:
av_dens_trapz = at.integrate_lambda_density(density_arrays, lam_vals)

av_dens_cspl = at.integrate_lambda_density(density_arrays, lam_vals, method='cspline')
print(av_dens_trapz.sum())
print(av_dens_cspl.sum())

In [4]:
# projection on line
x = np.linspace(0, 20, 175)
sa = (1,2)
av_dens_1D = av_dens_cspl.sum(axis=sa)
trpz = av_dens_trapz.sum(axis=sa)
pd_1D = [ve38.project(sa), ve30.project(sa), ve23.project(sa), ve15.project(sa), ve8.project(sa), ve0.project(sa), av_dens_1D, trpz]
label = [r'$\rho(\lambda = 1.0)$',r'$\rho(\lambda \approx 0.79)$', r'$\rho(\lambda \approx 0.61)$', r'$\rho(\lambda \approx 0.39)$', r'$\rho(\lambda \approx 0.21)$', r'$\rho(\lambda = 0)$', r'$\tilde{\rho} \approx \int_0^1 d\lambda \rho(\lambda, r)$', 'trpz']
fig, ax = plt.subplots(1,1)
for idx, pd in enumerate(pd_1D):
    # bigger linewidth for lambda-averaged density
    if idx == len(pd_1D)-1:
        ax.plot(x, pd, '--', label=label[idx])
    else:
        ax.plot(x, pd, label=label[idx])

ax.set_title('Boxsize = 20 Ang')
ax.set_xlabel('Cell coordinate (Ang)')
ax.set_xlim(0.5, 19.5)
ax.set_ylabel(r'Projected Density $\rho(x_0)$ (Bohr$^{-3}$) ')

ax.legend()

NameError: name 'av_dens_cspl' is not defined

In [9]:
from mpl_toolkits.mplot3d import Axes3D
X = np.array([x, x, x, x, x, x])
Y = np.array([ve0.project(sa), ve8.project(sa), ve15.project(sa), ve23.project(sa), ve30.project(sa), ve38.project(sa)])
Z = np.empty((6, 175))
for idx, val in enumerate(lam_vals):
    tmp = np.zeros(175)
    tmp.fill(val)
    Z[idx] = tmp

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(X, Z, Y)

In [ ]:
densities[3][86][86][87]
# np.where(densities[0]==np.amax(densities[0]))
f = np.empty(len(densities))
for idx in range(0, len(densities)):
    f[idx] = densities[idx][86][86][87]


In [ ]:
plt.plot(lam_vals,f, 'o')

In [ ]:
import scipy
interp= scipy.interpolate.CubicSpline(lam_vals, f, bc_type=('clamped', 'not-a-knot'))
interp.c

f2=interp.__call__(np.linspace(0, 1, 100))

In [ ]:
plt.plot(lam_vals,f, '-o')
plt.plot(np.linspace(0, 1, 100),f2)